## Install Libraries


In [1]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 41.5 MB/s eta 0:00:00


## Import Libraries


In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

#channel_1 = 16
#channel_2 = 32
#channel_3 = 64
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 2
pad_size_3 = 3

class Net(nn.Module):
    def __init__(self, channel_1, channel_2, channel_3, fc_count_1):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [6]:
def train_cifar(config):
    net = Net(
        config["channel_1"], 
        config["channel_2"], 
        config["channel_3"], 
        config["fc_count_1"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-2)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["channel_1"], 
        best_result.config["channel_2"], 
        best_result.config["channel_3"], 
        best_result.config["fc_count_1"] 
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"channel_1": tune.grid_search([16, 32]), 
          "channel_2": tune.grid_search([32, 64]), 
          "channel_3": tune.grid_search([64, 128]), 
          "fc_count_1": tune.grid_search([256, 512, 1024])}



In [9]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


2023-04-12 03:11:22,220	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=3313) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-11-21/train_cifar_b46fe_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-12_03-11-23/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 77088576.98it/s]


(train_cifar pid=3313) Extracting /root/ray_results/train_cifar_2023-04-12_03-11-21/train_cifar_b46fe_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-12_03-11-23/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-11-21/train_cifar_b46fe_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-12_03-11-23/data
(train_cifar pid=3313) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_b46fe_00000,0.689,2023-04-12_03-12-32,True,,2cf43835272e41c786e6f747516e772c,"0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256",ff63d4b5d613,10,0.932895,172.28.0.12,3313,True,65.8487,4.62774,65.8487,1681269152,0,,10,b46fe_00000,0.00379467
train_cifar_b46fe_00001,0.7163,2023-04-12_03-13-23,True,,2cf43835272e41c786e6f747516e772c,"1_channel_1=32,channel_2=32,channel_3=64,fc_count_1=256",ff63d4b5d613,10,0.853214,172.28.0.12,3313,True,50.3862,4.42828,50.3862,1681269203,0,,10,b46fe_00001,0.00379467
train_cifar_b46fe_00002,0.6943,2023-04-12_03-14-16,True,,2cf43835272e41c786e6f747516e772c,"2_channel_1=16,channel_2=64,channel_3=64,fc_count_1=256",ff63d4b5d613,10,0.939898,172.28.0.12,3313,True,52.7324,4.56856,52.7324,1681269256,0,,10,b46fe_00002,0.00379467
train_cifar_b46fe_00003,0.7162,2023-04-12_03-15-10,True,,2cf43835272e41c786e6f747516e772c,"3_channel_1=32,channel_2=64,channel_3=64,fc_count_1=256",ff63d4b5d613,10,0.893603,172.28.0.12,3313,True,53.9945,4.68687,53.9945,1681269310,0,,10,b46fe_00003,0.00379467
train_cifar_b46fe_00004,0.7217,2023-04-12_03-16-01,True,,2cf43835272e41c786e6f747516e772c,"4_channel_1=16,channel_2=32,channel_3=128,fc_count_1=256",ff63d4b5d613,10,0.830567,172.28.0.12,3313,True,50.8902,4.50099,50.8902,1681269361,0,,10,b46fe_00004,0.00379467
train_cifar_b46fe_00005,0.7258,2023-04-12_03-16-52,True,,2cf43835272e41c786e6f747516e772c,"5_channel_1=32,channel_2=32,channel_3=128,fc_count_1=256",ff63d4b5d613,10,0.83813,172.28.0.12,3313,True,51.0777,4.58532,51.0777,1681269412,0,,10,b46fe_00005,0.00379467
train_cifar_b46fe_00006,0.6965,2023-04-12_03-17-42,True,,2cf43835272e41c786e6f747516e772c,"6_channel_1=16,channel_2=64,channel_3=128,fc_count_1=256",ff63d4b5d613,10,1.01806,172.28.0.12,3313,True,50.5877,4.45498,50.5877,1681269462,0,,10,b46fe_00006,0.00379467
train_cifar_b46fe_00007,0.7398,2023-04-12_03-18-33,True,,2cf43835272e41c786e6f747516e772c,"7_channel_1=32,channel_2=64,channel_3=128,fc_count_1=256",ff63d4b5d613,10,0.817781,172.28.0.12,3313,True,50.8008,4.59445,50.8008,1681269513,0,,10,b46fe_00007,0.00379467
train_cifar_b46fe_00008,0.715,2023-04-12_03-19-24,True,,2cf43835272e41c786e6f747516e772c,"8_channel_1=16,channel_2=32,channel_3=64,fc_count_1=512",ff63d4b5d613,10,0.863342,172.28.0.12,3313,True,50.4339,4.49348,50.4339,1681269564,0,,10,b46fe_00008,0.00379467
train_cifar_b46fe_00009,0.7269,2023-04-12_03-20-17,True,,2cf43835272e41c786e6f747516e772c,"9_channel_1=32,channel_2=32,channel_3=64,fc_count_1=512",ff63d4b5d613,10,0.833251,172.28.0.12,3313,True,53.3888,4.67485,53.3888,1681269617,0,,10,b46fe_00009,0.00379467


...

(train_cifar pid=3313) Extracting /root/ray_results/train_cifar_2023-04-12_03-11-21/train_cifar_b46fe_00023_23_channel_1=32,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-12_03-31-38/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-11-21/train_cifar_b46fe_00023_23_channel_1=32,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-12_03-31-38/data
(train_cifar pid=3313) Files already downloaded and verified


2023-04-12 03:32:39,694	INFO tune.py:798 -- Total run time: 1276.22 seconds (1276.06 seconds for the tuning loop).


(train_cifar pid=3313) Finished Training
Best trial config: {'channel_1': 32, 'channel_2': 64, 'channel_3': 128, 'fc_count_1': 1024}
Best trial final validation loss: 0.8058406300605483
Best trial final validation accuracy: 0.7528


100%|██████████| 170498071/170498071 [00:03<00:00, 47805502.37it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.7013


In [10]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/channel_1,config/channel_2,config/channel_3,config/fc_count_1
23,0.805841,0.7528,4.680610,False,61.264386,32,64,128,1024
13,0.778107,0.7421,4.538289,False,55.924703,32,32,128,512
19,0.763572,0.7420,4.519915,False,41.991998,32,64,64,1024
3,0.781562,0.7415,4.500029,False,49.307617,32,64,64,256
7,0.817781,0.7398,4.594451,False,50.800844,32,64,128,256
5,0.762396,0.7376,4.476915,False,42.079248,32,32,128,256
10,0.803166,0.7365,4.532693,False,37.917809,16,64,64,512
15,0.791523,0.7338,4.628613,False,45.909672,32,64,128,512
11,0.798922,0.7328,4.782371,False,38.312752,32,64,64,512
21,0.805391,0.7313,4.605221,False,50.007736,32,32,128,1024


In [11]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/channel_1,config/channel_2,config/channel_3,config/fc_count_1
0,0.858184,0.7050,4.554220,False,56.634308,16,32,64,256
2,0.860657,0.7081,4.440805,False,43.558650,16,64,64,256
20,0.837036,0.7105,4.512931,False,41.655952,16,32,128,1024
6,0.855227,0.7147,4.542068,False,36.992557,16,64,128,256
16,0.819812,0.7157,4.517471,False,37.236852,16,32,64,1024
4,0.830567,0.7217,4.500992,False,50.890180,16,32,128,256
14,0.817557,0.7226,4.620557,False,37.195272,16,64,128,512
8,0.812301,0.7233,4.672443,False,45.940441,16,32,64,512
12,0.800122,0.7244,4.640646,False,38.436453,16,32,128,512
18,0.838326,0.7248,4.535942,False,51.208582,16,64,64,1024
